In [4]:
include("nn_ops.jl")

NNOps

In [5]:
batch = 1
in_height = 5
in_width = 5
stride_height = 2
stride_width = 2
pooled_height = round(Int, in_height/stride_height, RoundUp)
pooled_width = round(Int, in_width/stride_width, RoundUp)
in_channels = 1
filter_height = 2
filter_width = 2
out_channels = 4

srand(5)
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)
filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)
x_conv_actual = NNOps.conv2d(x_actual, filter)
x_conv_relu_actual = NNOps.relu(x_conv_actual)
x_conv_relu_maxpool_actual = NNOps.maxpool(x_conv_relu_actual, (1, 2, 2, 1));

In [6]:
using JuMP
using Gurobi

m = Model(solver=GurobiSolver())

@variable(m, ve[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx_conv[1:batch, 1:in_height, 1:in_width, 1:out_channels])
# @variable(m, vx_conv_relu[1:batch, 1:in_height, 1:in_width, 1:out_channels])
# @variable(m, vx_conv_relu_maxpool[1:batch, 1:pooled_height, 1:pooled_width, 1:out_channels])
@constraint(m, vx .== x_current) # input

# 1. Only using convolution constraint
vx_conv = NNOps.conv2dconstraint(m, vx+ve, filter)
# @constraint(m, conv2d(vx+ve, filter) .== vx_conv)
# @constraint(m, vx_conv .== x_conv_actual)

# 2. Adding relu layer
vx_conv_relu = NNOps.reluconstraint(m, vx_conv, 10000)
# @constraint(m, vx_conv_relu .== x_conv_relu_actual)

# 3. Adding maxpool layer
vx_conv_relu_maxpool = NNOps.maxpoolconstraint(m, vx_conv_relu, (2, 2), 10000)
@constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual)


@objective(m, Min, sum(ve.^2))

:Min

In [7]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(ve))

# TODO: fix the error message WARNING: A variable named a is already attached to this model. If creating variables programmatically, consider using the anonymous variable syntax x = @variable(m, [1:N], ...).

Optimize a model with 797 rows, 586 columns and 2209 nonzeros
Model has 25 quadratic objective terms
Variable types: 386 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 393 rows and 261 columns
Presolve time: 0.00s
Presolved: 404 rows, 325 columns, 1148 nonzeros
Presolved model has 25 quadratic objective terms
Variable types: 173 continuous, 152 integer (152 binary)
Found heuristic solution: objective 104293.81848
Found heuristic solution: objective 60920.897582

Root relaxation: objective 2.020828e+02, 531 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  202.08278    0   92 60920.8976  202.08278   100%     -    0s
H    0     0                    180

In [8]:
x_conv_current = NNOps.conv2d(x_current+getvalue(ve), filter)
x_conv_relu_current = NNOps.relu(x_conv_current)
x_conv_relu_maxpool_current = NNOps.maxpool(x_conv_relu_current, (1, 2, 2, 1));

In [9]:
maximum(abs(x_conv_current - x_conv_actual))

235.0000000000054

In [10]:
maximum(abs(x_conv_relu_current - x_conv_relu_actual))

123.00000000000543

In [11]:
maximum(abs(x_conv_relu_maxpool_current - x_conv_relu_maxpool_actual))

9.985257065636688e-10

In [ ]:
@printf "%d + %d" 5 3

In [ ]:
@printf "%s + %s" (1, 2, 3) (4, 5, 6)

In [ ]:
@nloops 4 r x_conv_actual begin
    print(@ntuple 4 r)
end

In [ ]:
prod((1, 2, 3, 4))

In [ ]:
methods(max)

In [ ]:
subtypes(Number)

In [ ]:
output_size = (1, 2, 3, 4)
hippo = reshape(@variable(m, [1:prod(output_size)]), output_size)
typeof(hippo)